# Encuesta Nacional de Nutrición y Salud (ENNyS2)

## Introducción

La Segunda Encuesta Nacional de Nutrición y Salud (ENNyS 2) es un relevamiento realizado en Argentina por la Secretaría de Gobierno de Salud de la Nación (hoy Ministerio de Salud) entre octubre de 2018 y febrero de 2019.

Su objetivo principal es describir la situación alimentaria y de salud de la población, recolectando información detallada sobre:

* Consumo de alimentos y bebidas (recordatorio de 24 horas, frecuencia y tipo de alimentos).

* Estado nutricional (mediciones antropométricas y datos de composición corporal).

* Enfermedades crónicas y factores de riesgo (hipertensión, diabetes, colesterol alto, etc.).

* Condiciones sociodemográficas y del hogar (cobertura de salud, nivel educativo, ingresos, etc.).

* Hábitos de vida (actividad física, lactancia, consumo de tabaco y alcohol, etc.).

La encuesta utiliza un diseño probabilístico, estratificado y multietápico, lo que permite que los resultados sean representativos a nivel nacional y por regiones.

### Se estructura en cuestionarios específicos por grupo etario:

* C1: Niños/as menores de 5 años.

* C2: Niños/as y adolescentes de 5 a 12 años.

* C3: Adolescentes de 13 a 17 años.

* C4: Adultos de 18 años o más.

En este trabajo utilizaremos la base principal de publicada en el portal de datos abiertos del Ministerio de Salud, junto con las bases complementarias de alimentos y nutrientes, para realizar un análisis que vincule alimentación, estado de salud y factores asociados.

## Objetivos de la notebook

Explorar la base principal de la ENNyS2 (encuesta) para:
- ubicar variables clave (edad, sexo, indicadores de salud, educación, región),
- revisar cobertura de datos en adultos (≥18),
- generar un subset mínimo limpio para notebooks posteriores.
- Mostramos capacidades de la base sin plantear hipótesis temáticas.

## Estructura de las columnas en ENNyS2

La base principal de ENNyS2 contiene aproximadamente 1830 columnas.
Estas variables se organizan en módulos y prefijos que indican el tipo de información, la población objetivo y, en algunos casos, la posición del miembro del hogar.

### Convenciones de nombres:

Cn_: Variables de cuestionarios específicos según grupo etario:

C1_: Niños/as menores de 2 años (respondido por adulto responsable).

C2_: Niños/as y adolescentes de 2 a 12 años.

C3_: Adolescentes de 13 a 17 años.

C4_: Adultos de 18 años o más.

T_Cn_: Variables totales o derivadas de un módulo.
Ejemplo: T_C4_ActFisTotal (total de minutos de actividad física en adultos).

I_Dx_Cn_: Indicadores por día dentro del módulo.
Ejemplo: I_D1_C4_Kcal (ingesta calórica del día 1 en adultos).

Mn_: Información de miembros del hogar (hasta 15 posibles).
Ejemplo: M1_Edad, M2_Sexo (edad y sexo del miembro 1 y 2).

---

## 1. Cargar la base CSV

In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA = Path("../data") # Previamente hay que descargar los datos manualmente y descomprimirlos en este directorio, mas detalles en el README.md
FILE = DATA / "ENNyS2_encuesta.csv"

df = pd.read_csv(FILE, encoding="utf-8-sig", low_memory=False) # Encoding para acentos

df.shape

(21358, 1830)

# Aclaración de cómo se realiza la encuesta

## 1. Formulario sociodemográfico (hogar e individuos)

* Todos los hogares seleccionados responden un formulario básico.

* Ahí se registran todos los integrantes: edad, sexo, parentesco, educación, variables socioeconómicas, etc.

* Esa es la información mínima, los llamados microdatos básicos.

## 2. Sorteo del individuo

* De cada hogar se elige un integrante al azar.

* Ese es el único que responde el cuestionario individual completo según su edad:

    * C1 → 0 a 23 meses

    * C2 → 2 a 12 años

    * C3 → 13 a 17 años

    * C4 → 18 años y más

**Importante**: solo el seleccionado tiene datos completos de salud, alimentación, antecedentes, etc.

## 3. Recordatorio de 24 horas (R24H)

* También se aplica solo al seleccionado.

* Registra todo lo consumido el día anterior (alimentos, bebidas, cantidades, horarios, contexto).

* En algunos casos se hace un segundo R24H en otro día, para captar variabilidad.

* Si no puede responder (bebés → se entrevista al adulto responsable, rechazos, etc.), el registro queda faltante.

* Estos datos se guardan en bases específicas:

    * Base de alimentos: cada fila es un alimento consumido.

    * Base de nutrientes: desglose de energía y nutrientes de esos alimentos.

## 4. Quienes no fueron sorteados

* El resto de los integrantes del hogar solo tienen los datos sociodemográficos básicos.

* No tienen cuestionario C1–C4 ni recordatorio de 24 hs.

## En la práctica de análisis

* Primero filtramos a los individuos con ES_Seleccionado = "Si", son los que efectivamente respondieron cuestionario individual + R24H. Son los de interes porque tienen informacion de alimentacion completa como para calcular el porcentaje de consumo de ultra procesados.

* Luego dividimos esos casos en los cuatro grupos etarios (C1, C2, C3, C4).

* Con eso trabajamos las variables de interés (ej. socioeconómicas, consumo, estado nutricional).

---

## 2. Separamos en 4 subbases (C1 a C4)


In [9]:
df_sel = df[df["ES_Seleccionado"]=="Si"]

df_c1 = df_sel[df_sel["E_CUEST"]=="0 a 23 meses"]
df_c2 = df_sel[df_sel["E_CUEST"]=="2 a 12 años"]
df_c3 = df_sel[df_sel["E_CUEST"]=="13 a 17 años"]
df_c4 = df_sel[df_sel["E_CUEST"]=="18 o mas años"]

print("0 a 23 meses:", df_c1.shape[0])
print("2 a 12 años:", df_c2.shape[0])
print("13 a 17 años:", df_c3.shape[0])
print("18 años y más:", df_c4.shape[0])

0 a 23 meses: 2013
2 a 12 años: 216
13 a 17 años: 401
18 años y más: 2876


In [16]:
# Función auxiliar
def columnas_por_completitud(df_sub, nombre):
    # calcula % de no-nulos por columna
    completeness = df_sub.notnull().mean().sort_values(ascending=False)
    print(f"\nSubbase {nombre} ({df_sub.shape[0]} registros)")
    print("-"*50)
    print(completeness.head(10))   # 10 con más datos
    print("\nColumnas casi vacías:")
    print(completeness.tail(10))   # 10 con menos datos
    return completeness

# Aplicar a cada subbase
comp_c1 = columnas_por_completitud(df_c1, "C1 (0 a 23 meses)")
comp_c2 = columnas_por_completitud(df_c2, "C2 (2 a 12 años)")
comp_c3 = columnas_por_completitud(df_c3, "C3 (13 a 17 años)")
comp_c4 = columnas_por_completitud(df_c4, "C4 (18 años y más)")



Subbase C1 (0 a 23 meses) (2013 registros)
--------------------------------------------------
activo_OMS_adol          1.0
id                       1.0
Srvyr                    1.0
Duration                 1.0
Upload                   1.0
Status                   1.0
HO_finde_min_nn          1.0
HO_semanal_tot_min_nn    1.0
HPO_semanal_min_nn       1.0
HPO_finde_min_nn         1.0
dtype: float64

Columnas casi vacías:
C2_V_2_4_OtherS         0.0
C2_PC_3_5_OtherS        0.0
C2_HAC_5_1_8_OtherS     0.0
C2_EE_7_5_OtherS        0.0
C3_SGA_1_2_OtherS       0.0
C2_HAC_5_3              0.0
C2_A_8_3                0.0
C2_AF_4_19_O4_OtherS    0.0
C2_V_2_7_OtherS         0.0
C1_PC_5_5_OtherS        0.0
dtype: float64

Subbase C2 (2 a 12 años) (216 registros)
--------------------------------------------------
activo_OMS_adol          1.0
id                       1.0
Srvyr                    1.0
Duration                 1.0
Upload                   1.0
Status                   1.0
HO_finde_min_nn

---

## 2. Seleccionamos columnas relevantes

In [4]:
# === 2. Columnas relevantes ===
columnas_relevantes = [
    # Datos básicos
    "Edadd", "C4_SEXO", "IMC", "Cobertura_salud", "IngHog_UC_QUINT_imp", "region",
    
    # Enfermedades crónicas
    "T_C4_SG_1_2_1",  # Diabetes
    "T_C4_SG_1_2_2",  # Enfermedad renal
    "T_C4_SG_1_2_3",  # Hipertensión o presión alta
    "T_C4_SG_1_2_4",  # Dislipemia, colesterol o triglicéridos altos
    "T_C4_SG_1_2_5",  # Anemia
    "T_C4_SG_1_2_6",  # Cardiopatía
    "T_C4_SG_1_2_7",  # Sonda vesical
    "T_C4_SG_1_2_8",  # Transplante de riñón
    "T_C4_SG_1_2_9",  # Obesidad
    "T_C4_SG_1_2_10", # EPOC
    "T_C4_SG_1_2_11", # Infarto
    "T_C4_SG_1_2_12", # Cáncer
    "T_C4_SG_1_2_13", # Leucemia
    "T_C4_SG_1_2_14"  # Otras enfermedades de larga duración
]

# === 3. Renombrar columnas ===
nombres_amigables = {
    "Edadd": "edad",
    "C4_SEXO": "sexo",
    "IMC": "imc",
    "Cobertura_salud": "cobertura_salud",
    "IngHog_UC_QUINT_imp": "quintil_ingresos",
    "region": "region",
    "T_C4_SG_1_2_1": "diabetes",
    "T_C4_SG_1_2_2": "enf_renal",
    "T_C4_SG_1_2_3": "hipertension",
    "T_C4_SG_1_2_4": "dislipemia",
    "T_C4_SG_1_2_5": "anemia",
    "T_C4_SG_1_2_6": "cardiopatia",
    "T_C4_SG_1_2_7": "sonda_vesical",
    "T_C4_SG_1_2_8": "transplante_renal",
    "T_C4_SG_1_2_9": "obesidad",
    "T_C4_SG_1_2_10": "epoc",
    "T_C4_SG_1_2_11": "infarto",
    "T_C4_SG_1_2_12": "cancer",
    "T_C4_SG_1_2_13": "leucemia",
    "T_C4_SG_1_2_14": "otras_enf_larga_duracion"
}

df_sel = df_ad[columnas_relevantes].rename(columns=nombres_amigables)

df_sel.head()


,edad,sexo,imc,cobertura_salud,quintil_ingresos,region,diabetes,enf_renal,hipertension,dislipemia,anemia,cardiopatia,sonda_vesical,transplante_renal,obesidad,epoc,infarto,cancer,leucemia,otras_enf_larga_duracion
976,1308,,,Obra social o prepaga,5,GBA,,,,,,,,,,,,,,
977,5737,,31.45034,Obra social o prepaga,4,GBA,,,,,,,,,,,,,,
980,5264,,17.09889,Obra social o prepaga,3,GBA,,,,,,,,,,,,,,
981,672,,18.91265,Obra social o prepaga,3,GBA,,,,,,,,,,,,,,
983,2100,,19.14394,Solo pública,4,GBA,,,,,,,,,,,,,,
